# Pathfinding Challenge

In [ ]:
import numpy as np
import pyvista as pv
import shapely as sh
import matplotlib.pyplot as plt

from airfoil.util.array_helpers import split_indexable
from airfoil.util.shapely_helpers import (
    plot_shapely_directional,
    plot_shapely
)
from airfoil.util.linestring_helpers import (
    deflection_angle,
    resample_spline_fallback_linear,
    ensure_closed,
    remove_sequential_duplicates,
    split_and_roll_at_top_right,
)
from airfoil.util.pyvista_helpers import create_ruled_surface

In [ ]:
fig, (
    (axa, axb, axc),
    (axd, axe, axf),
) = plt.subplots(2,3,figsize=(16,10))


# create components
s0 = sh.Point(0,0).buffer(60)
s1 = sh.Point(0,0).buffer(60+30)
s2 = sh.box(-100,-100,100,0)
s3 = sh.box(-15,50, 15,250)
plot_shapely_directional([s0,s1,s2,s3], ax=axa)
axa.set_title("components")


# use boolean operations to create spork shape
axb.set_title("polygon")
spork:sh.Polygon = sh.difference(sh.union(s3,sh.difference(s1,s2)),s0)
plot_shapely_directional([spork],legend=["spork"],ax=axb)


# split polygon at its corners into individual segments
axc.set_title("split at corners")
segments:list[np.ndarray] = split_indexable(
    spork.exterior.coords, 
    np.where(deflection_angle(spork.exterior.coords)>np.deg2rad(30))[0]+1
)
plot_shapely_directional([sh.LineString(segment) for segment in segments], ax=axc)


# resample the edges so that they have points at a nice consistent interval
axd.set_title("re-sampled")
resampled = [resample_spline_fallback_linear(segment, lambda total_length:np.ceil(total_length/5)) for segment in segments]
plot_shapely_directional([sh.LineString(segment) for segment in resampled], ax=axd)

# connect the segments back into a polygon
axe.set_title("recombined")
spork_edge = sh.Polygon(remove_sequential_duplicates(ensure_closed(np.concat(resampled))))
plot_shapely_directional([spork_edge], ax=axe)

#triangulate the polygon
axf.set_title("triangulated")
spork_triangulated = sh.MultiPolygon([triangle for triangle in sh.delaunay_triangles(spork_edge).geoms if spork_edge.contains(triangle.centroid)])
plot_shapely([spork_triangulated],ax=axf)

segment_lengths = [len(segment) for segment in resampled]
print(f"The number of points in each segment at the re-sampled stage were {segment_lengths}")

In [ ]:
fig, (
    (axa, axb, axc),
    (axd, axe, axf),
) = plt.subplots(2,3,figsize=(16,10))

hole = sh.Point(0,100).buffer(90)
spork2 = sh.intersection(spork,hole)
# some reversing and rolling is reqired in this case to ensure coordinates line up with the original
spork2 = sh.Polygon(split_and_roll_at_top_right(spork2)[::-1]) 
plot_shapely_directional([spork2],ax=axa)



segments = split_indexable(
    spork2.exterior.coords, 
    np.where(deflection_angle(spork2.exterior.coords)>np.deg2rad(30))[0]+1
)
plot_shapely_directional([sh.LineString(segment) for segment in segments],ax=axb)
resampled =[resample_spline_fallback_linear(segment, lambda _:count) for segment, count in zip(segments, segment_lengths)]
spork2_edge = sh.Polygon(remove_sequential_duplicates(ensure_closed(np.concat(resampled))))
plot_shapely_directional([spork2_edge],ax=axc)


axf.set_title("triangulated")
spork2_triangulated = sh.MultiPolygon([triangle for triangle in sh.delaunay_triangles(spork2_edge).geoms if spork2_edge.contains(triangle.centroid)])
plot_shapely([spork2_triangulated],ax=axf)

In [ ]:
spork2_edge_transformed = sh.affinity.translate(sh.affinity.rotate(spork2_edge,10),20,-10)
plot_shapely_directional([
    spork_edge,
    spork2_edge_transformed
])

spork2_triangulated_transformed = sh.affinity.translate(sh.affinity.rotate(spork2_triangulated,10),20,-10)

In [ ]:
# descide on the width of the foam we will cut from
foam_width = 150

# move to 3d by converting each triangle into a mesh
spork_mesh:pv.PolyData = pv.merge([pv.PolyData(np.insert(geom.exterior.coords[:-1],0,-foam_width/2,-1), faces=[[3, 0,1,2]]) for geom in spork_triangulated.geoms])
spork_mesh2:pv.PolyData = pv.merge([pv.PolyData(np.insert(geom.exterior.coords[:-1],0,foam_width/2,-1), faces=[[3, 0,1,2]]) for geom in spork2_triangulated_transformed.geoms])

# create a ruled surface between
ruled_surface = create_ruled_surface(
    np.insert(spork_edge.exterior.coords,0,-foam_width/2,axis=-1),
    np.insert(spork2_edge_transformed.exterior.coords,0,foam_width/2,axis=-1),
)

pt = pv.Plotter()
pt.add_mesh(spork_mesh, color="brown")
pt.add_mesh(spork_mesh2, color="teal")
pt.add_mesh(ruled_surface, edge_color='#222244', show_edges=True, opacity=0.5)

pt.show()

In [ ]:
# combined all the parts into a single mesh
combined_mesh = pv.merge([spork_mesh, spork_mesh2, ruled_surface])
assert combined_mesh.is_manifold
combined_mesh.plot_normals(faces=True, mag=10, show_edges=True, edge_color="grey")


In [ ]:
from airfoil.cnc.cnc_machine_mesh import axis

def state_to_3d_points(x:float, y:float, z:float, a:float, spacing=220):
    return (
        (-spacing/2,x,y),
        ( spacing/2,z,a),
    )

def draw_machine(x:float, y:float, z:float, a:float, spacing=220, pt:pv.Plotter|None=None):
    bottom_limit = (100,100)
    axa = axis(
        position=(-spacing/2,x,y),
        side="L",
        bottom_limit=bottom_limit,
    )
    axb = axis(
        position=(spacing/2,z,a),
        side="R",
        bottom_limit=bottom_limit,
    )
    if pt is None:
        pt = pv.Plotter()
    pt.add_mesh(axa,color="white")
    pt.add_mesh(axb,color="white")
    pt.add_mesh(pv.Line(*state_to_3d_points(x,y,z,a,spacing=spacing)),color="red")
    pt.camera_position = [
        (+spacing/2,-1000,300),
        (0,150,80),
        (0,0,1)
    ]
    return pt

pt = draw_machine(0,0,0,0)
pt.add_mesh(combined_mesh)
pt.add_floor(pad=1,color="thistle")

pt.show()